# Geocode Singapore Addresses

In [2]:
import pandas as pd
import glob
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [3]:
resale_price_all = glob.glob("./*.csv")

li = []

for filename in resale_price_all:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

resale_df = pd.concat(li, axis=0, ignore_index=True)
resale_df.head()
resale_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746207 entries, 0 to 746206
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                746207 non-null  object 
 1   town                 746207 non-null  object 
 2   flat_type            746207 non-null  object 
 3   block                746207 non-null  object 
 4   street_name          746207 non-null  object 
 5   storey_range         746207 non-null  object 
 6   floor_area_sqm       746207 non-null  float64
 7   flat_model           746207 non-null  object 
 8   lease_commence_date  746207 non-null  int64  
 9   resale_price         746207 non-null  float64
 10  remaining_lease      37153 non-null   float64
dtypes: float64(3), int64(1), object(7)
memory usage: 62.6+ MB


In [4]:
resale_df['ADDRESS'] = df['block'].astype(str) + ' ' + df['street_name'] + ',' + ' Singapore'   

resale_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,ADDRESS
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,NaN,"174 ANG MO KIO AVE 4, Singapore"
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,NaN,"541 ANG MO KIO AVE 10, Singapore"
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,NaN,"163 ANG MO KIO AVE 4, Singapore"
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,NaN,"446 ANG MO KIO AVE 10, Singapore"
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,NaN,"557 ANG MO KIO AVE 10, Singapore"


In [5]:
from geopy.extra.rate_limiter import RateLimiter
locator = Nominatim(user_agent="myGeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
resale_df['location'] = resale_df['ADDRESS'][:1000].apply(geocode)
resale_df['point'] = resale_df['location'][:1000].apply(lambda loc: tuple(loc.point) if loc else None)
resale_df[['latitude', 'longitude','altitude']] = pd.DataFrame(resale_df['point'].tolist(), index=resale_df.index)
resale_df.head()

KeyboardInterrupt: 

In [ ]:
resale_df = resale_df.drop(['block', 'street_name', 'point', 'location', 'altitude'], axis=1)
resale_df.head()

In [11]:
resale_df[:1000].to_csv('./resale_1k.csv', index=False)